# LIF Class

In [14]:
import math
import matplotlib.pyplot as plt


class LIF:

    def __init__(self, u_rest, resistance, tau, threshold,
                 is_adaptive, tau_w, a, b,
                 is_exponential, delta_tt, theta_rh):
        self.u_rest = u_rest
        self.resistance = resistance
        self.tau = tau
        self.threshold = threshold
        self.is_adaptive = is_adaptive
        self.tau_w = tau_w
        self.a = a
        self.b = b
        self.is_exponential = is_exponential
        self.delta_tt = delta_tt
        self.theta_rh = theta_rh
        self.u_t = []
        self.i_t = []
        self.w_t = []
        self.sigma_delta_func = 0
        self.u = u_rest
        self.w = 0
        self.spike_count = 0
    ## Action Function For Model
    
    def action(self, i, dt):
        exp_val = 0
        if self.is_exponential:
            exp_val = self.delta_tt * math.exp((self.u - self.theta_rh) / self.delta_tt)
        du = dt * (-(self.u - self.u_rest) + exp_val - self.resistance * self.w + self.resistance * i) / self.tau
        self.u += du

        if self.is_adaptive:
            dw = (dt / self.tau_w) * (self.a * (self.u - self.u_rest)
                                      - self.w + self.b * self.tau_w * self.sigma_delta_func)
            self.w += dw
        self.sigma_delta_func = 0

        if self.u >= self.threshold:
            self.u = self.u_rest
            self.sigma_delta_func = 1
            self.spike_count += 1

        self.u_t.append(self.u)
        self.w_t.append(self.u)
        self.i_t.append(i)


In [4]:

def work(model, func_i, time, dt):
    current_t = 0
    while current_t <= time:
        model.action(func_i(current_t), dt)
        current_t += dt

# Ploting

In [15]:
 def i_t_plot(i_t, dt):
    plt.plot(list(map(lambda i: i * dt, range(len(i_t)))), i_t)
    plt.ylabel('I')
    plt.xlabel('Time')
    plt.title('I-T plot')


def w_t_plot(w_t, dt):
    plt.plot(list(map(lambda i: i * dt, range(len(w_t)))), w_t)
    plt.ylabel('w')
    plt.xlabel('Time')
    plt.title('W-T plot')


def u_t_plot(u_t, dt):
    plt.plot(list(map(lambda i: i * dt, range(len(u_t)))), u_t)
    plt.ylabel('U')
    plt.xlabel('Time')
    plt.title('U-T plot')


def f_i_plot(i_t, time, dt, ma):
    m_i = list(set(i_t))
    f_i = []

    for i in m_i:
        def func(t):
            return i + 0 * t

        mdl = LIF(u_rest=ma['u_rest'], resistance=ma['r'], tau=ma['tau'], threshold=ma['threshold'],
                  is_adaptive=ma['is_a'], tau_w=ma['tau_w'], a=ma['a'], b=ma['b'],
                  is_exponential=ma['is_e'], delta_tt=ma['delta_T'], theta_rh=ma['theta_rh'])
        work(mdl, func, time, dt)
        f_i.append(mdl.spike_count / time)
    plt.plot(m_i, f_i)
    plt.ylabel('f')
    plt.xlabel('I')
    plt.title('F-I plot')

